In [26]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [27]:
# Define data generator to load images efficiently
# datagen = ImageDataGenerator(rescale=1.0/255)  # Normalize pixel values
# Create DataGenerator for training & validation with automatic split
datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize images (0-1)
    validation_split=0.2  # 20% of data for validation
)
# lec05Cnn/

In [40]:
# Enable shuffling for better training
train_generator = datagen.flow_from_directory(
    "project/wonder_classifier/WondersofWorld",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="training",
    shuffle=True  # Ensure data is shuffled
)

val_generator = datagen.flow_from_directory(
    "project/wonder_classifier/WondersofWorld",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="validation",
    shuffle=False  # No need to shuffle validation data
)


Found 3082 images belonging to 12 classes.
Found 764 images belonging to 12 classes.
Epoch 1/2
97/97 ━━━━━━━━━━━━━━━━━━━━ 82s 847ms/step - accuracy: 0.4630 - loss: 1.5870 - val_accuracy: 0.5563 - val_loss: 1.3368
Epoch 2/2
97/97 ━━━━━━━━━━━━━━━━━━━━ 85s 879ms/step - accuracy: 0.6017 - loss: 1.2457 - val_accuracy: 0.6715 - val_loss: 1.0421


In [34]:
# Define CNN model
model = Sequential([
    # Convolutional layers
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2,2)),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    # Flatten feature maps to fully connected layers
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),  # Prevent overfitting
    Dense(128, activation='relu'),
    Dense(12, activation='softmax')  # 12 classes → softmax for multi-class classification
])

In [35]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),  # Adam optimizer
    loss="categorical_crossentropy",  # Multi-class classification
    metrics=["accuracy"]  # Track accuracy
)

In [36]:
# Print model summary
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 256)            │    22,151,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 12)             │         1,548 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,279,116 (84.99 MB)

 Trainable params: 22,279,116 (84.99 MB)

 Non-trainable params: 0 (0.00 B)

In [37]:
# Train the model using the generators
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=2,  # Adjust as needed
)

Epoch 1/2
97/97 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - accuracy: 0.1835 - loss: 2.4499 - val_accuracy: 0.3730 - val_loss: 1.9990
Epoch 2/2
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 278us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00


2025-03-15 23:53:30.287820: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
2025-03-15 23:53:30.303163: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [18]:
import tensorflow as tf

# Check if GPU is available
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

# Print GPU details
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    for gpu in gpu_devices:
        print("GPU Found:", gpu)
else:
    print("No GPU found, running on CPU.")


Num GPUs Available: 0
No GPU found, running on CPU.
